# Criptografía y Computación
## Práctica 1 

<div align="right"><b>Autor</b>: Jorge Gangoso Klöck
<div align="right"><b>Fecha</b>: 10/03/2022
    

### Ejercicio 1
<b>Implementa el algoritmo extendido de Euclides para el cálculo del máximo común divisor: dados
dos enteros a y b, encuentra u, v ∈ Z tales que au + bv es el máximo común divisor de a y b. </b>

La función debe encontrar los coeficientes de bézout que son los últimos valores de u y v al realizar el algoritmo Extendido de Euclides. Los valores de u y v se deben inicializar de forma que $a*(u=1)+b*(v=0) = a$ y de forma análoga $a*(u=0)+b*(v=1) = b$.

In [19]:
# Función que calcula el MCD de dos números (a y b) y también devuelve los coeficientes de bézout en una misma tupla
def MCD(a,b):
    (u0,u1) = (1,0)
    (v0,v1) = (0,1)
    while(b > 0):
        (c, r) = (a//b, a%b)
        (u0, u1) = (u1, u0-c*u1)
        (v0, v1) = (v1, v0-c*v1)
        a = b
        b = r
    return [a, u0, v0]

In [3]:
# Ejemplo de ejecución
a = 21651
b = 54445689
[mcd, bz1, bz2] = MCD(a,b)
print('El MCD es ', mcd, 'Los coeficientes de bezout son: ', bz1, ' y ', bz2)
print('Por tanto: (', bz1, ' * ', a, ') + (', bz2, ' * ', b, ') = ', bz1*a+bz2*b)

El MCD es  3 Los coeficientes de bezout son:  3171032  y  -1261
Por tanto: ( 3171032  *  21651 ) + ( -1261  *  54445689 ) =  3


### Ejercicio 2
<b>Usando el ejercicio anterior, escribe una función que calcule $a^{-1} \bmod b$ para cualesquiera a, b enteros que sean primos relativos.</b>

Se asume una entrada de datos correctos en los que a y b son primos relativos. En caso de que no lo sea el algoritmo devolverá siempre 1 ya que el algoritmo Extendido de Euclides para en la primera iteración con cociente $b/a$ y resto $0$. En ese caso el inverso realmente no es 1 si no que no existe. Se podría añadir una comprobación sencilla si $b \bmod a = 0$ entonces devolver que a no tiene inverso módulo b.

In [4]:
# Función que obtiene el inverso cogiendo el coeficiente de bézout correspondiente del algoritmo Extendido de Euclides
def InversoModular(a, b):
    [a, bz1, bz2] = MCD(a,b)
    return bz1%b
def InversoModularComprobante(a, b):
    if b%a == 0:
        return False
    else:
        [a, bz1, bz2] = MCD(a,b)
        return bz1%b

In [5]:
# Ejemplo de ejecución
    # 1 - Inverso Modular de números primos relativos
print(InversoModularComprobante(7, 48))
    # 2 - Inverso Modular de múltiplos sin comprobación (22245 = 1483 * 15)
print(InversoModular(1483, 22245))
    # 3 - Inverso Modular de múltiplos con comprobación (22245 = 1483 * 15)
print(InversoModularComprobante(1483, 22245))

7
1
False


### Ejercicio 3

<b>Escribe una función que calcule $a^{b} \bmod n$ para cualesquiera a, b y n enteros positivos. La implementación debería tener en cuenta la representación binaria de b.</b>

Separamos el exponente b en su forma binaria como suma de potencias de 2. Cada bit activo en su interpretación binaria implica elevar $a^{2^{i}}$ pero puesto que esto es lo mismo que hacer $a^{2i}$ podemos sencillamente ir actualizando $a = a^2$ lo que nos da un valor de $a = a^i$ para la iteración i-ésima y actualizar la variable que va almacenando el valor total de la potencia si y sólo si $b_i = 1$ es decir, el bit que correspondería con dicha iteración está activo en la representación binaria de b.

In [6]:
# Función que calcula una potencia módulo n con un algoritmo de complejidad logarítmica
def PotenciaModular(a, b, n):
    p = 1
    b = int(b)
    while(b > 0):
        if(b%2 == 1):
            p = (p*a)%n
        b = b>>1 #Es lo mismo que (b-b%2)/2
        a = (a*a)%n
    return p

In [7]:
# Ejemplo de ejecución
print(PotenciaModular(382711723928839982739,2837193,78))

51


### Ejercicio 4
<b> Dado un entero p, escribe una función para determinar si p es (probablemente) primo usando el
método de Miller-Rabin. </b>

Para realizar el ejercicio de la forma más legible y encapsulada posible he realizado dos funciones, la primera inicializa los parámetros y llama K veces (K=10 para una probabilidad de error de $\frac{1}{4^{10}}$) a la segunda función llamada listaMillerRabin que hace la comprobación de si el número es probablemente primo o no lo es utilizando la potenciación modular y la condición necesaria (aunque no suficiente) de que si un cuadrado es 1, su raíz debe ser 1 o -1, en caso contrario el número nunca será primo. Si ningún cuadrado es 1 también podemos afirmar que el número no es primo.

In [8]:
# Biblioteca: random -> para escoger un número aleatorio entre 2 y n-2 como base para la lista 
import random

# Función que comprueba si dada la descomposición de un número como p-1=(2^u)*s devuelve si p es probablemente primo
# Entrada: n(número del que se comprueba la primalidad), u(potencia de 2 de la descomposición), s(número impar).
def listaMillerRabin (n, u, s):
    a = random.randint(2,n-2)
    previo = n-1
    actual = PotenciaModular(a, s, n)
    for i in range (0, u+1):
        if actual == 1 and previo == n-1:
            return True
        if actual == 1 and previo != n-1:
            return False
        previo = actual
        actual = PotenciaModular(actual, 2, n)
    return False

# Función general para Miller-Rabin. Inicializa parámetros y realiza la llamada al verdadero test de primalidad.
def MillerRabin (n):
    if n>0 and n<4:
        return True
    if n%2 == 0:
        return False
    else:
        u = 0
        s = n-1
        #Calcular u
        while s%2 == 0:
            u = u+1
            s = s/2
        #Tenemos s
        #Hacer 10 iteraciones a listaMillerRabin
        for i in range (10):
            primo = listaMillerRabin(n, u, s)
            if primo == False:
                return False
        return True


In [9]:
#Ejemplo de Ejecución
print(MillerRabin(123456789101119)) #Es primo
print(MillerRabin(8432487)) #No es primo

True
False


### Ejercicio 5
<b>Implementa el algoritmo paso enano-paso gigante para el cálculo de logaritmos discretos en $\mathbb{Z}_{p}$</b>

Consiste en encontrar un valor $x$ tal que $x = log_{a}(b) \bmod n$. Por tanto como entradas de la función tendremos la base a, el número al que queremos calcularle el logaritmo, y la base n. Nótese que no siempre existirá el $log_{a}(b) \bmod n$, pero siempre que sea calculable este algoritmo lo calcula.
El algoritmo consiste en crear dos listas, una que crece rápidamente con la forma: $ a^{uc} \bmod n$ y otra que crece más lentamente con la forma: $ ba^u \bmod n $. Si se encuentra un elemento común en ambas listas quiere decir que existe una solución a nuestro problema que se puede escribir como $ x = cu-v : u \in \{1,...,c\}, v \in \{0,...,c-1\}$.

In [10]:
import math

# Función para hacer el LogaritmoDiscreto tal que a^k = b (mod n)
# Entrada: a(Base), b(Número del que queremos calcular el logaritmo), n(módulo)
def LogaritmoDiscreto (a, b, n, info):
    c = 1+math.isqrt(n-2)
    u = 1
    v = 0
    big_steps = []
    small_steps = []
    for i in range (0,c):
        big_steps.append(PotenciaModular(a, (u+i)*c, n))
        small_steps.append((b*PotenciaModular(a, (v+i), n))%n)
        
    # Creamos diccionarios para recuperar rápidamente los índices
    big_dict = dict((k,i) for i,k in enumerate(big_steps))   
    small_dict = dict((k,i) for i,k in enumerate(small_steps))
        
    if len(set(big_dict).intersection(small_steps)) == 0:
        return 'No existe'
    
    # Usamos los diccionarios para obtener los valores de u y v
    intersection_b = set(big_dict).intersection(small_steps)
    u = [ big_dict[x]+1 for x in intersection_b ]
    v = [ small_dict[x] for x in intersection_b ]
    
    if info:
        print('C: ',c,'V: ', v,'U: ', u)
        print('Paso pequeño {0...c-1}: ', small_steps)
        print('Paso gigante {1...c}: ', big_steps)

    # Finalmente nos quedamos sólo con el primer valor de coincidencia
    return c*(u[0]) - v[0]

In [11]:
# Ejemplo de ejecución 
base = 10
valor = 14
modulo = 31
resultado = LogaritmoDiscreto(base, valor, modulo, True)
print('log en base ', base, ' de ', valor, ' modulo ', modulo, ': ', resultado)
base = 10
valor = 21
modulo = 31
resultado = LogaritmoDiscreto(base, valor, modulo, False)
print('log en base ', base, ' de ', valor, ' modulo ', modulo, ': ', resultado)

C:  6 V:  [1, 4] U:  [4, 2]
Paso pequeño {0...c-1}:  [14, 16, 5, 19, 4, 9]
Paso gigante {1...c}:  [2, 4, 8, 16, 1, 2]
log en base  10  de  14  modulo  31 :  23
log en base  10  de  21  modulo  31 :  No existe


### Ejercicio 6
<b>Sea $n = pq$, con $p$ y $q$ enteros primos positivos.</b>
1. <b>Escribe una función que, dado un entero $a$ y un primo $p$ con $(\frac{a}{p}) = 1$, devuelve $r$ tal que $r^2 ≡ a \bmod p$</b>
2. <b>Sea $a$ un entero que es residuo cuadrático módulo $p$ y $q$. Usa el teorema chino de los restos para calcular
todas las raíces cuadradas de $a \bmod n$ a partir de las raíces cuadradas de $a$ módulo $p$ y $q$.</b>

En primer lugar implementamos el símbolo de Jacobi que dado un número a y un primo p tenemos que $(\frac{a}{p}) = 1$ si y sólo si $a^{\frac{p-1}{2}}  ≡ 1 \bmod p$ y de forma análoga tenemos que $(\frac{a}{p}) = -1$ si y sólo si $a^{\frac{p-1}{2}}  ≡ -1 \bmod p$. En cualquier otro caso devuelve $0$.

A continuación implementamos el cálculo de raíces cuadradas módulo un primo. Para poder hallar este valor es necesario que $(\frac{a}{p}) = 1$ por tanto se añade una comprobación inicial de que esto ocurre y una comprobación adicional de que p es primo. A continuación procedemos con el algoritmo, expresamos el número $p-1$ como potencia de 2 multiplicada por un número forzosamente impar $s$, inicializamos $r = a^{\frac{s+1}{2}} \bmod p$. También por eficiencia ya que va a ser un número que va a utilizarse en cada iteración almacenamos en una variable el valor de $a^{-1}$ ya que será constante a lo largo de toda la ejecución.

Lo último que debemos preparar antes del bucle es encontrar un valor $m$ tal que $(\frac{m}{p}) = -1$ y nos quedamos con su cuadrado al cual llamaremos $\mu = m^{2}$

Para terminar el bucle que comprueba si $(a^{-1}*r^2)^{2^{n-i}} = 1 \lor -1$ En el caso que vale $1$ que incluye al caso base $(a^{-1}*r^2)^{2^{n-1}} = 1$ no tenemos que hacer nada más que elevar $\mu^2$, tras eso como estamos haciendo el paso opuesto a elevar al cuadrado sólo podemos obtener $1$ y $-1$. En el caso $-1$ debemos rectificar el símbolo de la raíz haciendo $r = r*\mu$ y una vez más actualizamos $\mu^2$. Así hasta la última iteración en la que obtenemos la raíz.

En ambos casos tanto con el cálculo de raíces cuadradas de primos como de compuestos podemos observar como dependiendo del valor aleatorio con el que se comienza el algoritmo podemos obtener una raíz o su opuesta, en el caso del segundo algoritmo es más notorio ya que casi en todas las diferentes ejecuciones cambia el orden en que se muestran las raíces.

In [12]:
# Función auxiliar que calcula el símbolo de Jacobi de (a,p)
def Jacobi(a, p):
    return PotenciaModular(a, (p-1)//2, p)

# Función que cálcula la raíz cuadrada de a módulo un primo p tal que Jaocbi(a,p) = 1
def RaizCuadradaModularPrimo(a, p):
    if (MillerRabin(p) != True):
        return 'P no es primo'
    if (Jacobi(a,p) != 1):
        return 'Jacobi(a,p)!=1'
    u = 0
    s = p-1
    #Calcular u y s
    while s%2 == 0:
        u = u+1
        s = s//2
        
    if u == 1:
        return PotenciaModular(a, (p + 1)//4, p)
    
    r = PotenciaModular(a, (s+1)//2, p)
    a_inv = InversoModular(a, p)
    
    ### Calcular un m tal que Jacobi(m, p) = -1. Con P=1/2
    for i in range (p-2):
        m = random.randint(2,p-1)
        if (Jacobi(m,p)==p-1):
            break
    if(Jacobi(m,p) != p-1):
        return 'No se ha encontrado m con Jacobi(m,p)=-1'
    
    nu = PotenciaModular(m, s, p)
    
    for i in range (u-2, -1, -1):
        tmp1 = PotenciaModular(r,2,p)
        temp = PotenciaModular(tmp1*a_inv, pow(2,i), p)
        if (temp == 1):
            nu = PotenciaModular(nu, 2, p)
        elif (temp == p-1):
            r = (r*nu)%p
            nu = PotenciaModular(nu, 2, p)
    return r
    

In [13]:
# Ejemplo de ejecución 
print('La raíz cuadrada de 24 módulo 71 es: ', RaizCuadradaModularPrimo(24, 71))
print('La raíz cuadrada de 24 módulo 70 es: ', RaizCuadradaModularPrimo(24, 70))
print('La raíz cuadrada de 23 módulo 71 es: ', RaizCuadradaModularPrimo(23, 71))
print('La raíz cuadrada de 4 módulo 5 es: ', RaizCuadradaModularPrimo(4, 5))


La raíz cuadrada de 24 módulo 71 es:  38
La raíz cuadrada de 24 módulo 70 es:  P no es primo
La raíz cuadrada de 23 módulo 71 es:  Jacobi(a,p)!=1
La raíz cuadrada de 4 módulo 5 es:  2


In [14]:
def RaizCuadradaModular(a, p, q):
    rp = RaizCuadradaModularPrimo(a, p)
    rq = RaizCuadradaModularPrimo(a, q)
    solucion = []
    x = (rp + (p*InversoModular(p, q)*(rq-rp))%(p*q))%(p*q)
    solucion.append(x)
    solucion.append((p*q)-x)
    rp = (p*q)-rp
    x = (rp + (p*InversoModular(p, q)*(rq-rp))%(p*q))%(p*q)
    solucion.append(x)
    solucion.append((p*q)-x)
    return solucion

In [15]:
# Ejemplo de ejecución
print('Las raices cuadradas de 4 módulo 15 son: ', RaizCuadradaModular(4, 3, 5))

Las raices cuadradas de 4 módulo 15 son:  [7, 8, 2, 13]


### Ejercicio 7
1. <b>Implementa el Método de Fermat para factorización de enteros.</b>
2. <b>Implementa el algoritmo de factorización ρ de Pollard.</b>

#### 7.1
Para el ejercicio 1 he implementado una función auxiliar y una función que extiende ligeramente la funcionalidad del algoritmo básico, aunque no lo agiliza.

    Función Cuadrado
La función auxiliar cuadrado la he creado para comprobar si un número es un cuadrado perfecto o no. Simplemente toma la raíz entera por abajo de un número y comprueba si su cuadrado es el número original. En caso afirmativo tenemos un cuadrado, en caso contario no.
    
    Función FactorizacionFermat
Es el algoritmo básico para encontrar factores de Fermat. Se trata de un algoritmo sumamente lento en los peores casos que toma un número inicial e itera intentando encontrar un número que sea cuadrado y que además se corresponda con el valor de $a*a-n$. Cuando lo encuentra la raíz de ese cuadrado es un factor propio del número inicial. Es muy importante notar que este algoritmo encuentra factores que no tienen por qué ser primos necesariamente.

    Función FactorizacionFermatCompleta
Este algoritmo no es más que un burdo esquema de cómo se podría utilizar el algoritmo básico de factorización para factorizar un número en todos sus factores, sin embargo como se ha comentado anteriormente, ya que el algoritmo básico de Fermat no garantiza encontrar factores primos, la factorización no tiene tampoco por qué estar compuesta únicamente por factores primos. Por supuesto podría modificarse de manera relativamente sencilla de forma que cada vez que se encuentre un factor, se compruebe si es primo y en caso contrario se llame recursivamente a la función de factorización para así obtener la factorización completa como producto de primos, pero ya que no es el objetivo del ejercicio y no tiene mayor complicación se deja explicado sólamente.

#### 7.2
Para el ejercicio 2, implementamos el algoritmo Rho de Pollard para factorización de enteros, un algoritmo mucho más rápido que el de Fermat y que está basado en la idea de la Liebre y la Tortuga utilizando una función (Normalmente de la forma: $x^2+c \bmod n, c \neq 0,-2$) que se aplica una vez sobre el valor Tortuga y se aplica dos veces sobre el valor Liebre. Tras haber aplicado la función se comprueba el $MCD(|x-y|, n)$ y este puede tomar tres valores:
1. $1$ en cuyo caso se continua con el algoritmo
2. $n$ en cuyo caso se detiene el algoritmo y se devuelve un fallo que puede implicar que n sea primo pero puede también ser un falso error, no implica necesariamente la ausencia de factores
3. Un valor distinto de $1 \lor n$ en cuyo caso ya hemos encontrado un factor que es precisamente el MCD

In [16]:
import numpy as np
def Cuadrado(n):
    r_n = math.isqrt(n)
    if (r_n*r_n == n):
        return True
    else:
        return False

def FactorizacionFermat(n):
    a = 1+math.isqrt(n-1)
    b_cuadrado = (a*a)-n
    while (Cuadrado(b_cuadrado) == False):
        a = a+1
        b_cuadrado = a*a-n
    factor = a - math.isqrt(b_cuadrado)
    return factor

def FactorizacionFermatCompleta(n):
    m = []
    while (MillerRabin(n) == False):
        factor = FactorizacionFermat(n)
        n = n//factor
        m.append(factor)
    m.append(n)
    return m
        

def f(x,n):
    return (x**2 + 3)%n

def FactorizacionPollard(n):
    x = 2
    y = 2
    d = 1
    while (d == 1):
        x = f(x,n)
        y = f(f(y,n),n)
        d = MCD(np.abs(x-y),n)[0]
    if d == n:
        return False
    else:
        return d
    

In [17]:
# Ejemplo de ejecución
print('Factorizacion Fermat: ', FactorizacionFermat(35456515))
#print('Factorizacion Fermat Multifactor: ',FactorizacionFermatCompleta(35456515))
print('Factorizacion Rho: ', FactorizacionPollard(35456515))

Factorizacion Fermat:  5
Factorizacion Rho:  5


### Ejercicio 8
<b>Compara los tiempos de ejecución de tus implementaciones con las de tus compañeros y con las
primitivas de algunos paquetes de cálculo simbólico como (GAP, MATHEMATICA, maxima, . . . ).</b>

Para este ejercicio usaré la tabla de cálculo que hemos creado en el grupo de la asignatura para comparar mis tiempos con los de mis compañeros de clase.

In [18]:
import time, collections
 
tiempos = collections.defaultdict(int)
EJECUCIONES = 1000
 
EXTRA = [46381, 768479]
for _ in range(EJECUCIONES):
    t = time.time()
    MCD(EXTRA[0], EXTRA[1])
    tiempos['gcd'] += time.time() - t

    t = time.time()
    InversoModular(EXTRA[0], EXTRA[1])
    tiempos['modinv'] += time.time() - t

    t = time.time()
    PotenciaModular(EXTRA[0], EXTRA[0], EXTRA[1])
    tiempos['modpow'] += time.time() - t

    t = time.time()
    MillerRabin(EXTRA[0])
    tiempos['Miller-Rabin'] += time.time() - t

    aux = PotenciaModular(51, 79, EXTRA[0])
    t = time.time()
    LogaritmoDiscreto(51, aux, EXTRA[0], False)
    tiempos['enano-gigante'] += time.time() - t

    aux = PotenciaModular(EXTRA[0], 2, EXTRA[1])
    t = time.time()
    RaizCuadradaModularPrimo(aux, EXTRA[1])
    tiempos['Tonelli-Shanks'] += time.time() - t

    n = EXTRA[0] * EXTRA[1]
    aux = PotenciaModular(57, 2, n)
    t = time.time()
    RaizCuadradaModular(aux, EXTRA[0], EXTRA[1])
    tiempos['CRT'] += time.time() - t

    t = time.time()
    FactorizacionPollard(n)
    tiempos['Pollard'] += time.time() - t

for i in tiempos: 
    tiempos[i] /= EJECUCIONES
 # Fermat va mucho más lento, así que lo hago con menos ejecuciones
for _ in range(25):
    t = time.time()
    FactorizacionFermat(n)
    tiempos['Fermat'] += time.time() - t
tiempos['Fermat'] /= 25
 
for i in tiempos:
    print(i+':', tiempos[i])

KeyboardInterrupt: 

In [22]:
print(MCD(15,31))
print(MCD(31, 127))
print(MCD(15,127))

[1, -2, 1]
[1, 41, -10]
[1, 17, -2]
